# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.4020427180880297

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
6.6e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0023 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00011 s/step ...done
MCMC, adapting ϵ (200 steps)
3.8e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
6.6e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
8.3e-6 s/step ...done
MCMC (1000 steps)
6.8e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([-0.392354], -3.08442, 1, AdjacentTurn, 0.999288, 3), NUTS_Transition{Array{Float64,1},Float64}([0.0550203], -2.9579, 1, AdjacentTurn, 0.981495, 3), NUTS_Transition{Array{Float64,1},Float64}([0.161337], -2.91172, 1, DoubledTurn, 0.978167, 1), NUTS_Transition{Array{Float64,1},Float64}([0.240918], -3.01444, 1, DoubledTurn, 0.978598, 1), NUTS_Transition{Array{Float64,1},Float64}([0.141264], -2.98112, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.162928], -2.93921, 1, DoubledTurn, 0.994852, 1), NUTS_Transition{Array{Float64,1},Float64}([1.38846], -6.27734, 1, AdjacentTurn, 0.490507, 3), NUTS_Transition{Array{Float64,1},Float64}([1.59187], -7.20011, 1, DoubledTurn, 0.880366, 1), NUTS_Transition{Array{Float64,1},Float64}([1.538], -7.42601, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.937787], -6.17141, 1, DoubledTurn, 1.0, 1)  …  NUTS_Transition{Array{Float64,

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

333.1589803962446

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.23 0.91 0.97 1.0 1.0
  termination: AdjacentTurn => 34% DoubledTurn => 66%
  depth: 1 => 62% 2 => 38%


check the mean

In [11]:
mean(posterior_α)

0.4452020626251314

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*